In [1]:
import os
import tensorflow as tf
from util import constants
from util.config_util import get_model_params, get_task_params, get_train_params
from tf2_models.trainer import Trainer
from absl import app
from absl import flags
import numpy as np
from util.models import MODELS
from util.tasks import TASKS
from notebook_utils import *

%matplotlib inline
import pandas as pd
import seaborn as sns; sns.set()

from tqdm import tqdm


In [6]:
task = 'sst2'

chkpt_dir='../tf_ckpts'
task = TASKS[task](get_task_params(), data_dir='../data')
cl_token = task.sentence_encoder().encode(constants.bos)
print(task.sentence_encoder().encode(constants.bos))
print(task.sentence_encoder().encode(constants.eos))
print(task.sentence_encoder().encode(constants.pad))
print(task.sentence_encoder().encode(constants.unk))
print(task.sentence_encoder().encode('this is good'))

print(task.sentence_encoder().encode(' '))

[3]
[4]
[1]
[2]
[23, 14, 280]
[8055]


In [7]:
config = {'model_name':'cl_bert',
        'model_config':'small_gpt_v9',
        'learning_rate':0.001,
        'exp_name':'test',
        'chkpt_dir': '../tf_ckpts'
}
hparams=get_model_params(task, config['model_name'], config['model_config'])
hparams.output_attentions = True
hparams.output_embeddings = True

cl_bert, ckpt = get_model(config, task, hparams, cl_token)

model config: small_gpt_v9
{'embedding_dim': 128, 'resid_pdrop': 0.4, 'embd_pdrop': 0.2, 'attn_pdrop': 0.6, 'initializer_range': 0.05}
No checkpoint found ../tf_ckpts/sst2/cl_bert_h-128_d-6_rdrop-0.4_adrop-0.6_indrop-0.2_small_gpt_v9_0.001_test


In [8]:
for x1,y1 in task.valid_dataset:
    y = cl_bert(x1)
    break

In [9]:
with tf.GradientTape() as tape:
    y = cl_bert(x1)

grads = tape.gradient(y, x1)

In [10]:
print(grads)

None


In [11]:
input_embeddings, input_shape, padding_mask, past = cl_bert.get_input_embeddings(x1)

In [12]:
print(x1.shape, input_embeddings.shape)

(64, 57) (64, 58, 128)


In [13]:
with tf.GradientTape() as tape:
    tape.watch(input_embeddings)
    outputs = cl_bert.call_with_embeddings(input_embeddings, input_shape, padding_mask, past)
    logits = outputs[0]
    
grads = tape.gradient(logits, input_embeddings)
print(grads.shape)

(64, 58, 128)
